In [9]:
# coding: utf-8
from typing import Dict, List

import pandas as pd


class Datos:
    datos: int
    nominalAtributos: List[bool]

    # Constructor: procesar el fichero para asignar correctamente las variables nominalAtributos, datos y diccionarios
    def __init__(self, nombreFichero: str):
        self.datosCrudos = pd.read_csv(nombreFichero)
        self.datos = self.datosCrudos.copy()

        self.nominalAtributos = []
        self.diccionarios = {}

        for columna in self.datos.columns:
            if self._es_nominal(columna):
                self.nominalAtributos.append(True)
                self.diccionarios[columna] = self._generar_mapeo(columna)
                self.datos[columna] = self._reemplazar_valores(columna)
            elif self._es_numerico(columna):
                self.nominalAtributos.append(False)
                self.diccionarios[columna] = {}
            else:
                raise ValueError(
                    f"La columna '{columna}' contiene valores que no son nominales ni enteros/decimales."
                )

    def _es_nominal(self, columna: str) -> bool:
        # es verdadero si es la columna objetivo o si sus valores son nominales
        return columna.lower() == "class" or self.datos[columna].dtype.name == "object"

    def _es_numerico(self, columna: str) -> bool:
        # es verdadero si los valores son un n�meros enteros o reales
        return self.datos[columna].dtype.name in ["int64", "float64"]

    def _generar_mapeo(self, columna_nominal: str):
        # se extraen los valores �nicos de la columna y se sortean lexicograficamente
        valores = [str(valor) for valor in self.datos[columna_nominal].unique()]
        valores = sorted(valores)

        return {valor: indice for indice, valor in enumerate(valores)}

    def _reemplazar_valores(self, columna_nominal: str) -> pd.Series:
        mapeo = self.diccionarios[columna_nominal]
        return self.datos[columna_nominal].map(lambda valor: mapeo[str(valor)])

    # Devuelve el subconjunto de los datos cuyos �ndices se pasan como argumento
    def extraeDatos(self, idx: List[int]):
        return self.datos.iloc[idx]


In [51]:
import random
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
datos = Datos("wdbc.csv").datos



#print("w actualizada: ", actualiza_w(umbral, datos.head(1)))

#print(np.array(datos.head(1))[0])

class ClasificadorRegresionLogistica:

    def __init__(self, datos, k_apre):
        self.datos = datos
        self.k_apre = k_apre
        self.umbral = [random.uniform(-0.5, 0.5) for _ in datos.columns[:-1]]
    
    # normalizar los datos
    for col in datos.columns:
        datos[col] = datos[col] / datos[col].abs().max()

    umbral = [random.uniform(-0.5, 0.5) for _ in datos.columns[:-1]]
    #k_apre = 1 # constante de aprendizaje
    
    def sigmoid(self, w_v, x_v):
        x = np.array(x_v)[0]
        w = np.array(w_v)[0]
        return 1 / (1 + np.exp(-w*x))


    def actualiza_w(self, w, x):
        t = x.iloc[-1]
        #print(t)
        x = np.array(x.iloc[:-1])
        return w - x * (self.sigmoid(w, x) - t) * self.k_apre
    
    def maximar_verosimilitud(self, epocas, w):
        verosimilitud_clase1 = 0
        for _ in range(0, epocas):
            for _, row in datos.iterrows():
                #print("ji")
                verosimilitud_clase1 = self.sigmoid(w, row)
                w = self.actualiza_w(w, row)
                print(verosimilitud_clase1)
        return verosimilitud_clase1
    

#print(datos.head(1))
#test = ClasificadorRegresionLogistica(datos.head(1), 1).actualiza_w(umbral_test, datos)
#print(test)

umbral = [random.uniform(-0.5, 0.5) for _ in datos.columns[:-1]]
clasi = ClasificadorRegresionLogistica(datos, 1)
print(clasi.maximar_verosimilitud(1, umbral))
#print(datos.iloc[3,:-1])


class ClasificadorRegresionLogisticaSkLearn:
    atributos = datos.iloc[:, :-1]
    objetivo = datos[:, -1]

    X_train, X_test, y_train, y_test = train_test_split(atributos, objetivo, test_size=0.2, random_state=42)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    modelo = LogisticRegression()
    modelo.fit(X_train_scaled, y_train)
    predicciones = modelo.predict(X_test_scaled)

0.5522234607275216
0.6105035708205387
0.6524552364555241
0.6140117419570607
0.7186850369098693
0.6604861426641667
0.74518639950857
0.7082368307846352
0.7123283383080545
0.7166746801203745
0.7798424988806675
0.7885546355620345
0.8428976823094118
0.8099080929412555
0.7871617725948927
0.8082739852910276
0.8182414314150025
0.846518016069323
0.8965239897441796
0.819196591345278
0.7817637452323655
0.6908594781145635
0.7632283701764795
0.8470693201159001
0.8045747356742544
0.8216201422785641
0.7950523499280898
0.8582390212847677
0.8222316266378809
0.8604757606074764
0.8794042287350234
0.7852190612014266
0.8719453962233177
0.9024732003281056
0.8699837394359474
0.8825837562477583
0.852272430209053
0.837154704543322
0.8424468078146247
0.8246177901820257
0.8297096315865311
0.7894980175851234
0.9135515038047983
0.8415263862916628
0.8443393586969672
0.9200356355118362
0.7481942558951667
0.838579208876198
0.823463483202746
0.8255800617177247
0.7666515865551615
0.772771310372322
0.7134494198230679
0.